In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

: 

In [ ]:
cd API

: 

In [ ]:
iris = load_iris()

: 

In [ ]:
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                  columns= iris['feature_names'] + ['target'])

: 

In [ ]:
X = df.iloc[:100,[0,2]].values
y = df.iloc[0:100,4].values
y = np.where(y == 0, -1, 1)

: 

In [ ]:
import matplotlib.pyplot as plt

: 

In [ ]:
plt.scatter(X[:50,0],X[:50,1],color='red', marker='o',label='setosa')
plt.scatter(X[50:100,0],X[50:100,1],color='blue', marker='x',label='versicolor')
plt.xlabel('sepal length (cm)')
plt.ylabel('petal length (cm)')
plt.legend(loc='upper left')
plt.show()

: 

In [ ]:
# implementacja 
class Perceptron():
    
    def __init__(self, eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
    
    def fit(self, X, y):
        self.w_ = np.zeros(1+X.shape[1])
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X,y):
                update = self.eta*(target-self.predict(xi))
                self.w_[1:] += update*xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def net_input(self, X):
        return np.dot(X, self.w_[1:])+self.w_[0]
    
    def predict(self, X):
        return np.where(self.net_input(X)>=0.0,1,-1)

: 

In [ ]:
# uzycie jak wszsytkie klasy sklearn
ppn = Perceptron(n_iter=20)
ppn.fit(X,y)

: 

In [ ]:
print(ppn.errors_)
print(ppn.w_)

: 

In [ ]:
ppn.predict(X)

: 

In [ ]:
ppn.predict([5.1,3.1])

: 

In [ ]:
# dodatkowa funkcja

from matplotlib.colors import ListedColormap

def plot_decision_regions(X,y,classifier, resolution=0.02):
    markers = ('s','x','o','^','v')
    colors = ('red','blue','lightgreen','gray','cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    x1_min, x1_max = X[:,0].min() - 1, X[:,0].max()+1
    x2_min, x2_max = X[:,1].min() -1, X[:,1].max()+1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(),xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl,0], y=X[y==cl,1], alpha=0.8, c=cmap(idx), marker=markers[idx], label=cl)

# dla kwiatków

: 

In [ ]:
plot_decision_regions(X,y,classifier=ppn)
plt.xlabel("dlugosc dzialki [cm]")
plt.ylabel("dlugosc platka [cm]")
plt.legend(loc='upper left')
plt.show()

: 

In [ ]:
def make_pretty(func):
    def inner():
        print("decorator działa")
        func()
    return inner()

: 

In [ ]:
def test():
    print("abc")

: 

In [ ]:
make_pretty(test)

: 

In [ ]:
@make_pretty
def test2():
    print("test2")

: 

In [ ]:
@make_pretty
def test3():
    print("jeszcze cos")

: 

# SERVER FLASK 

In [ ]:
import pickle

: 

In [ ]:
ppn.w_

: 

In [ ]:
### Zapisz swój najlepszy model 
with open('model.pkl', "wb") as picklefile:
    pickle.dump(ppn, picklefile)

: 

In [ ]:
with open('model.pkl',"rb") as picklefile:
    model = pickle.load(picklefile)

: 

In [ ]:
model.w_

: 

In [ ]:
model.predict([5.1,3.1])

: 

In [ ]:
import subprocess
import requests

: 

In [ ]:
!pip install flask

: 

In [ ]:
%%file app1.py

from flask import Flask

# Create a flask
app = Flask(__name__)

# Create an API end point
@app.route('/hello', methods=['GET'])
def say_hello():
    return "Hello World"

@app.route('/', methods=['GET'])
def say_he():
    return "Hello z innej strony"


if __name__ == '__main__':
    app.run()

: 

In [ ]:
p = subprocess.Popen(["python", "app1.py"]) #w cmd znajdziesz link do aplikacji, a jest tez w dockerze

: 

In [ ]:
p.kill()

: 

In [ ]:
response = requests.get("http://127.0.0.1:8080/hello")
response.content

: 

In [ ]:
%%file app.py

from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello():
    return "<h1>hello world</h1>"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

: 

In [ ]:
%%file requirements.txt
Flask==2.1.0

: 

In [ ]:
%%file Dockerfile

FROM python:3.10-slim-bullseye # odpalenie systemu 

WORKDIR /app #stworzenie katologu z aplikacja

COPY app.py .
COPY requirements.txt . #wymagane do aplikacji biblioteki

RUN pip install -r requirements.txt 

EXPOSE 5000 #port 5000 do nasluchiwania
#tu run nie dajemy, bo run tylko jak odpalamy aplikacje i ona sie konczy a to ponizej jak ta apka sobie ciagiem idzie
ENTRYPOINT ["python"]

CMD ["app.py"]

#popnizej zbudowanie i odpalenie dockera, kropka na koncu oznacza poszukaj sobie w biezacym katalogu. ten docker run odpalic w cmd

: 

```bash
docker build -t test_hello .

docker run -p 5000:5000 test_hello
```

In [ ]:
%%file app2.py

from flask import Flask
from flask import request

# Create a flask
app = Flask(__name__)

# Create an API end point
@app.route('/hello', methods=['GET'])
def say_hello():
    name = request.args.get("name", "")
    title = request.args.get("title", "")
    if name:
        resp = f"Hello {title} {name}" if title else f"Hello {name}"
    else:
        resp = f"Hello {title}" if title else "Hello"
    return resp

if __name__ == '__main__':
    app.run()

: 

In [ ]:
p = subprocess.Popen(["python", "app2.py"])

: 

In [ ]:
response = requests.get("http://127.0.0.1:5000/hello")
response.content

: 

In [ ]:
response = requests.get("http://127.0.0.1:5000/hello?name=Seba&title=mr")
response.content

: 

In [ ]:
params={"name": "Patryk", "title": "mr"}

: 

In [ ]:
response = requests.get("http://127.0.0.1:5000/hello", params=params)
response.content

: 

In [ ]:
p.kill()

: 

In [ ]:
%%file app.py

import pickle
from math import log10

from flask import Flask
from flask import request # w args przechowuje wszystkie zmienne wyslane jako get i post (jako string)
from flask import jsonify
import numpy as np

class Perceptron():
    
    def __init__(self, eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
    
    def fit(self, X, y):
        self.w_ = np.zeros(1+X.shape[1])
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X,y):
                update = self.eta*(target-self.predict(xi))
                self.w_[1:] += update*xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def net_input(self, X):
        return np.dot(X, self.w_[1:])+self.w_[0]
    
    def predict(self, X):
        return np.where(self.net_input(X)>=0.0,1,-1)

# Create a flask
app = Flask(__name__)

# Create an API end point
@app.route('/api/v1.0/predict', methods=['GET'])
def get_prediction():

    # sepal length
    sepal_length = float(request.args.get('sl'))
    petal_length = float(request.args.get('pl'))
    
    features = [sepal_length,
                petal_length]

    # Load pickled model file
    with open('model.pkl',"rb") as picklefile:
        model = pickle.load(picklefile)
        
    # Predict the class using the model
    predicted_class = int(model.predict(features))
    
    # Return a json object containing the features and prediction
    return jsonify(features=features, predicted_class=predicted_class) #features to to co wyprodukowal uzytkownik

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

: 

In [ ]:
!head app.py

: 

In [ ]:
!python app.py #mozna odpalic w taki sam sposob a terminalu, jak zrobimy tutaj to zablokujemy notatnik

: 

In [ ]:
# http://127.0.0.1:5000/api/v1.0/predict?&sl=4.5&pl=1.3

# pamiętaj otworzyć nowy notebook !
import requests
response = requests.get("http://127.0.0.1:5000/api/v1.0/predict?&sl=4.5&pl=1.3")
print(response.content) #odpalamy w przegladarce z drugiego pliku FLASK

: 

: 

: 